In [1]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 3

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

Using TensorFlow backend.
Saving vectors of label - 'happy': 100%|██████████| 1742/1742 [00:05<00:00, 330.67it/s]


In [2]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [3]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 3112 samples, validate on 2076 samples
Epoch 1/50
3112/3112 [==============================] - 2s 532us/step - loss: 1.4140 - acc: 0.4200 - val_loss: 1.0125 - val_acc: 0.4422
Epoch 2/50
3112/3112 [==============================] - 0s 146us/step - loss: 0.9526 - acc: 0.5562 - val_loss: 0.7775 - val_acc: 0.6763
Epoch 3/50
3112/3112 [==============================] - 0s 146us/step - loss: 0.7673 - acc: 0.6722 - val_loss: 0.5033 - val_acc: 0.8213
Epoch 4/50
3112/3112 [==============================] - 0s 146us/step - loss: 0.6052 - acc: 0.7487 - val_loss: 0.4076 - val_acc: 0.8560
Epoch 5/50
3112/3112 [==============================] - 0s 147us/step - loss: 0.5353 - acc: 0.7921 - val_loss: 0.5033 - val_acc: 0.8039
Epoch 6/50
3112/3112 [==============================] - 1s 165us/step - loss: 0.4731 - acc: 0.8220 - val_loss: 0.3199 - val_acc: 0.8892
Epoch 7/50
3112/3112 [==============================] - 0s 150us/step - loss: 0.4048 - acc: 0.8458 - val_loss: 0.4085 - val_acc: 0.8468


## Prediction

In [5]:
print(predict('./data/bed/004ae714_nohash_1.wav', model=model))

bed
